---
title: 9.4 Graph Theory - Introduction
subject:  Symmetric Matrices
subtitle: 
short_title: 9.4 Graph Theory - Introduction
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: 
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/08_Ch_9_Symmetric_Matrices/104-Graph.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 17 - Introduction to Graph Theory and Consensus Protocols.pdf>`

## Reading

Material related to this page can be found in [](https://murray.cds.caltech.edu/images/murray.cds/1/1e/Eeci-sp09_L4_graphtheory.pdf)

## Learning Objectives

By the end of this page, you should know:
- what is a consensus problem and its key components
- how a consensus problem is used to model the evolution of multiple agents in real life applications
- the formal definition of a graph and an example highlighting the key components
- connectedness of graphs
- the definition of different matrices used in graph theory

## Introduction to Graph Theory and Consensus

Many applications in engineering involve coordinating groups of agents to behave in a desirable way. Examples abound in a wide range of areas:
* Transportation: air traffic control and intelligent transportation systems
* Military: distributed aperture imaging and battlespace management
* Scientific: animal coordination and group opinion dynamics (e.g. on social networks)
* Sensor networks: adaptive ocean sampling, building sensor networks in green buildings
* General networks: communication, power, and supply chain networks

Many, but not all, of these problems can be posed as _consensus_ problems. In this case, we assume there are $n$ agents, with _each agent state_ $x_i(t) \in \mathbb{R}$ evolving according to the following differential equation:

\begin{equation}
\label{diff_con}
\dot{x}_i = f_i(x_i) + \sum_{j \in N_i} u_i(x_i, x_j), \quad i = 1,\ldots,n
\end{equation}

In [](#diff_con):
* the function $f_i: \mathbb{R} \to \mathbb{R}$ defines the _dynamics of agent $i$_, i.e., how the internal state of agent $i$ evolves in the absence of other agents.
* the _neighbor set $N_i$_ of agent $i$ are those agents that are directly connected to agent $i$, as specified by a connection graph $G$ with nodes $i = 1,\ldots,n$ corresponding to agents, and edges defining inter-agent communication.
* the _coordination rule_ $u_i: \mathbb{R} \times \mathbb{R} \to \mathbb{R}$, which we are to design so that the collective state of the agents $\vv x(t) := (x_1(t),\ldots,x_n(t))$ converges to a desired goal state $\vv x^*$.

In many settings of interest, each agent is a computer that is trying to compute or estimate a shared value. This is a common model adopted in distributed optimization and learning (want to compute a shared prediction), sensor networks (compute a shared estimate), and opinion dynamics (compute shared opinion). In these settings, it makes sense to set the local dynamics function $f_i(x) = 0$, so that an agent's state is determined solely by interactions with its neighbors. Agent $i$'s dynamics then become

\begin{equation}
\label{diff_con2}
\dot{x}_i = \sum_{j \in N_i} u_i(x_i, x_j), \quad i = 1,\ldots,n
\end{equation}

A common goal for such systems is for all agents to converge to the **same** value, i.e., that eventually $x_1 = x_2 = \cdots = x_n = m$, for some value $m$. If this happens, system [](#diff_con2) is said to _achieve consensus_ with _consensus value $m$_.

Studying the behavior of system [](#diff_con2) will require us to bring together all of the ideas that we've seen over the past few lectures on eigenvalues, dynamical systems, and spectral decomposition of symmetric matrices. We start with a summary of relevant facts from _graph theory_, which will help us model the flow of information in [](#diff_con2).

## A Primer on Graph Theory

We have used graphs before, but we pause now to describe formal linear algebraic representations thereof.

:::{prf:definition}
:label: graph_defn
We begin with the basic definition of a _graph_, which is defined as a pair $G = (\mathcal{V}, \mathcal{E})$ that consists of a set of _vertices_ $\mathcal{V}$ and a set of _edges_ $\mathcal{E} \subseteq \mathcal{V} \times \mathcal{V}$. A vertex $v_i \in \mathcal{V}$ is a node in the graph, and an edge $e_{ij} = (v_i, v_j) \in \mathcal{E}$ is an arc connecting node $i$ to node $j$. (Note: sometimes this notation is reversed.)
:::

::::{prf:example} An example graph
:label: graphs-ex1

:::{figure}../figures/09-graph.png
:label:graph
:alt:Graph
:width: 200px
:align: center
:::

$\mathcal{V} = \{1, 2, 3, 4, 5, 6\}$

$\mathcal{E} = \{(1,6), (2,1), (2,3), (2,6), (6,2), (3,4),(3,6), (4,3), (4,5), (5,1), (6,1), (6,2), (6,4)\}$

Path $\pi$ with $V_\pi = \{v_5, v_1, v_6, v_2\}$ and 
$E_\pi = \{(v_5,v_1), (v_1,v_6), (v_6,v_2)\}$ (see definition below)
::::

Some important notation and terminology for graphs include:

* The _order of a graph_ is the number of nodes $|\mathcal{V}|$
* Nodes $v_i$ and $v_j$ are _adjacent_ if there exists $e = (v_i, v_j) \in \mathcal{E}$ (i.e., if node $i$ is directly connected to node $j$ via an edge $e$)
* An adjacent node $v_j$ for a node $v_i$ is called a _neighbor_ of $v_i$
* The _set of all neighbors_ of $v_i$ is denoted by $N_i$
* A graph $G$ is called _complete_ if all nodes are adjacent


We will focus our study on the class of _undirected graphs_, that is graphs satisfying the property that if $e_{ij} \in \mathcal{E}$ then $e_{ji} \in \mathcal{E}$. In words, this means that all edges are bidirectional: if $v_j$ is a neighbor of $v_i$, then $v_i$ is a neighbor of $v_j$. Although we won't focus on them, a graph that is not undirected is called _directed_.

:::{note} Graph Connectivity
A fundamental question associated with the study of the consensus system [](#diff_con2) is under what properties of information sharing between agents can we guarantee that consensus is achieved. Intuitively, if each agent talks to "enough" other agents, we expect consensus to happen. This intuition can be formalized through the notion of _graph connectivity_.
:::

## Connectedness of Graphs

We start by defining a _path_ in $G$, which is a subgraph $\pi = (\mathcal{V}_\pi, \mathcal{E}_\pi)$, with distinct nodes $\mathcal{V}_\pi = \{v_1, v_2, ..., v_m\}$ and $\mathcal{E}_\pi = \{(v_1,v_2),(v_2,v_3),...,(v_{m-1},v_m)\}$. The _length_ of a path $\pi$ is defined as $|\mathcal{E}_\pi| = m-1$. For example, in the [graph above](#graph), the path $\pi$ defined by $\mathcal{V}_\pi = \{v_5, v_1, v_6, v_2\}$ and $\mathcal{E}_\pi = \{(v_5,v_1),(v_1,v_6),(v_6,v_2)\}$ is the path that goes from node 5 to node 1 to node 6 to node 2. It has length 3, the number of edges traversed. An undirected graph $G$ is called _connected_ if there exists a path $\pi$ between any two distinct nodes of $G$.

::::{prf:example} 
:label: graphs-ex2

[These graphs](#connected) are connected (all edges are bidirectional)

:::{figure}../figures/09-connected.png
:label:connected
:alt:connected Graph
:width: 400px
:align: center
:::

[This graph](#disconnected), composed of two disconnected subgraphs, is not:

:::{figure}../figures/09-disconnected.png
:label:disconnected
:alt:disconnected Graph
:width: 300px
:align: center
:::
::::

## Matrices Associated with a Graph

In our study of network flow problems, we encountered the incidence matrix associated with a graph. This is but one matrix we can associate with a graph $G$, and while the incidence matrix is convenient for encoding flow conservation, we will see that the _Graph Laplacian Matrix_ is more natural for defining consensus algorithms. 

:::{prf:definition} Adjacency Matrix
:label: adj_mat
We begin by defining the _adjacency matrix_ $A \in \mathbb{R}^{n \times n}$ of graph $G$ of order $n$ by

$$
a_{ij} = \begin{cases}
1 & \text{if } (v_i, v_j) \in \mathcal{E}, \quad i,j=1,\ldots,n \\
0 & \text{otherwise}
\end{cases}
$$
:::

:::{prf:definition} Degree Matrix
:label: deg_mat
The _degree matrix_ $\Delta$ of a graph $G$ is a diagonal $n \times n$ matrix with diagonal elements $\Delta_{ii}$ specified by the number of edges leaving node $i$, also called the _out degree of $v_i$_, which we'll denote by out$(v_i)$:

$$
\Delta_{ii} = \text{out}(v_i), \quad i=1,\ldots,n
$$
:::

:::{prf:definition} Laplacian Matrix
:label: lap_mat
The _Laplacian matrix_ $L$ of a graph is defined as $L = \Delta - A$. An important property of the Laplacian is that its rows all sum to zero, and that if a graph is undirected, then its adjacency matrix and its Laplacian are both symmetric.
:::

::::{prf:example} Adjacency and Laplacian matrices
:label: graphs-ex3

The adjacency matrix and Laplacian for the directed graph [above](#graph) are:

$$
A = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 \\
1 & 0 & 1 & 0 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 0 & 1 & 0 & 0
\end{bmatrix}
\quad
L = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 \\
-1 & 3 & -1 & 0 & 0 & -1 \\
0 & 0 & 2 & -1 & 0 & -1 \\
0 & 0 & -1 & 2 & -1 & 0 \\
-1 & 0 & 0 & 0 & 1 & 0 \\
-1 & -1 & 0 & -1 & 0 & 3
\end{bmatrix}
$$

The adjacency and Laplacian for the [udirected graph](#undirected)

:::{figure}../figures/09-undirected.png
:label:undirected
:alt:undirected Graph
:width: 300px
:align: center
:::

are

$$
& \quad \quad \begin{matrix}
1 & 2 & 3 & 4 \end{matrix} \\
A &= \begin{bmatrix}
0 & 1& 1 & 0  \\
1 & 0 & 1 & 0  \\
1 & 1 & 0 & 1  \\
0 & 0 & 1 & 0
\end{bmatrix}
\quad \text{and} \quad
L &= \begin{bmatrix}
2 & -1 & -1 & 0  \\
-1 & 2 & -1 & 0  \\
-1 & -1 & 3 & -1  \\
0 & 0 & 1 & 1
\end{bmatrix}
$$
::::

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/08_Ch_9_Symmetric_Matrices/104-Graph.ipynb)